<a href="https://colab.research.google.com/github/KevinNyagi/Bank-app-from-custom-library/blob/master/Unsupervised_Ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [43]:
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [44]:
import requests
import io
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import tensorflow_text as text

In [45]:
tf.config.run_functions_eagerly(True)

In [46]:

# Set the URL to download the PDF file from
URL = "https://www.nou.edu.ng/coursewarecontent/ENG%20223%20ADVANCED%20ENGLISH%20COMPOSITION%20I.pdf"


# Download the PDF file
response = requests.get(URL)

# Extract the text data from the PDF file
pdf_data = response.content
pdf_text = pdf_data.decode(encoding='latin-1')

# Clean the text data
text = re.sub(r'[^\w\s]', '', pdf_text)  # Remove special characters
text = re.sub(r'\d', '', text)  # Remove digits
text = text.lower()  # Lowercase the text
text = text.strip()  # Remove leading and trailing white space




In [56]:
# Tokenize the words
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
sequences = tokenizer.texts_to_sequences([text])

# Remove stop words
stop_words = ['a', 'an', 'and', 'the', 'in', 'on', 'at']
sequences = [[word for word in sequence if word not in stop_words] for sequence in sequences]

# Create a vocabulary of the most common words
word_counts = {}
for sequence in sequences:
    for word in sequence:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1
vocab = sorted(word_counts, key=word_counts.get, reverse=True)[:10000]

# Create sequences of input and output data
data = []
for i in range(len(sequences)):
    input_sequence = sequences[i][:-1]
    output_sequence = sequences[i][-1]
    data.append((input_sequence, output_sequence))

# Split the data into training and validation sets
#train_data, val_data = train_test_split(data, test_size=0.1)

# Split the data into training and validation sets with a manually specified train size
train_size = int(4.8 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]


In [57]:
!pip install tensorflow-hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [76]:
# Pad the input sequences to the same length
max_length = max([len(sequence) for sequence, _ in data])
train_inputs = pad_sequences([sequence for sequence, _ in train_data], maxlen=max_length)
val_inputs = pad_sequences([sequence for sequence, _ in val_data], maxlen=max_length)

# Create one-hot encoded outputs
num_words = len(vocab)
train_outputs = tf.one_hot([output for _, output in train_data], num_words)
val_outputs = tf.one_hot([output for _, output in val_data], num_words)

# Define the model architecture
inputs = tf.keras.layers.Input(shape=(max_length,))
x = tf.keras.layers.Embedding(num_words, 200)(inputs)

#Choose either a Recurrent Neural Network (RNN) or a Transformer model
use_rnn = True
if use_rnn:
  x = tf.keras.layers.LSTM(200)(x)
else:
  import tensorflow_hub as hub
  # Define the model architecture
  inputs = tf.keras.layers.Input(shape=(max_length,))
  x = tf.keras.layers.Embedding(num_words, 128)(inputs)

  # Use the TFBertModel layer from tensorflow_hub
  x = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1", trainable=True)(x)

outputs = tf.keras.layers.Dense(num_words, activation='softmax')(x)

#Compile the model with the Adam optimization algorithm and sparse categorical cross-entropy loss function
model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'],run_eagerly=True)
#model.compile(optimizer='adam', loss='categorical_crossentropy', run_eagerly=True)



In [77]:
#Define a function to create a vocabulary of the most common words
def create_vocab(data, num_words):
  word_counts = {}
  for sequence in data:
    for word in sequence:
      if word in word_counts:
        word_counts[word] += 1
      else:
        word_counts[word] = 1
  return sorted(word_counts, key=word_counts.get, reverse=True)[:num_words]


In [78]:
#Define a function to create sequences of input and output data
def create_sequences(data, vocab):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(vocab)
  sequences = tokenizer.texts_to_sequences(data)
  return sequences

In [79]:
#Define a function to split the data into training and validation sets
def split_data(data, test_size):
  return train_test_split(data, test_size=test_size)

In [80]:
#Define a function to train and evaluate the model on the data
def train_and_evaluate(model, train_inputs, train_outputs, val_inputs, val_outputs):
  
  model.fit(train_inputs, train_outputs, epochs=100, batch_size=100, validation_data=(val_inputs, val_outputs))
  _, train_accuracy = model.evaluate(train_inputs, train_outputs, return_dict=True)
  _, val_accuracy = model.evaluate(val_inputs, val_outputs, return_dict=True)
  

  return train_accuracy, val_accuracy


In [81]:
tf.debugging.enable_check_numerics()
#Train and evaluate the model
train_accuracy, val_accuracy = train_and_evaluate(model, train_inputs, train_outputs, val_inputs, val_outputs)
print(f'Train accuracy: {train_accuracy:.4f}')
print(f'Validation accuracy: {val_accuracy:.4f}')

Epoch 1/100


ValueError: ignored